# Importação de bibliotecas

In [1]:
#importacao bibliotecas
import numpy as np
import pandas as pd
import boto3

from io import BytesIO
from io import StringIO 

# Conexão com MinIO/S3

In [2]:
# cliente para se conectar ao MinIO
client = boto3.client('s3', 
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id='pyRqGqESCkanVco9',
    aws_secret_access_key='TB6ovlBFPCFeYr7rTSIQnGKCyFenm3Sc',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

# Leitura dos dataframes em CSV

In [3]:
# leitura do csv "estados.csv"
df_estados = pd.read_csv('estados.csv')
df_estados

,codigo_uf,uf,nome,latitude,longitude,regiao
0,11,RO,Rondônia,-10.83,-63.34,Norte
1,12,AC,Acre,-8.77,-70.55,Norte
2,13,AM,Amazonas,-3.47,-65.10,Norte
3,14,RR,Roraima,1.99,-61.33,Norte
4,15,PA,Pará,-3.79,-52.48,Norte
5,16,AP,Amapá,1.41,-51.77,Norte
6,17,TO,Tocantins,-9.46,-48.26,Norte
7,21,MA,Maranhão,-5.42,-45.44,Nordeste
8,22,PI,Piauí,-6.60,-42.28,Nordeste
9,23,CE,Ceará,-5.20,-39.53,Nordeste


In [4]:
df_estados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   codigo_uf  27 non-null     int64  
 1   uf         27 non-null     object 
 2   nome       27 non-null     object 
 3   latitude   27 non-null     float64
 4   longitude  27 non-null     float64
 5   regiao     27 non-null     object 
dtypes: float64(2), int64(1), object(3)
memory usage: 1.4+ KB


In [5]:
# leitura do csv "municipios"
df_municipios = pd.read_csv('municipios.csv')
df_municipios.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo


In [6]:
df_municipios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   codigo_ibge   5570 non-null   int64  
 1   nome          5570 non-null   object 
 2   latitude      5570 non-null   float64
 3   longitude     5570 non-null   float64
 4   capital       5570 non-null   int64  
 5   codigo_uf     5570 non-null   int64  
 6   siafi_id      5570 non-null   int64  
 7   ddd           5570 non-null   int64  
 8   fuso_horario  5570 non-null   object 
dtypes: float64(2), int64(5), object(2)
memory usage: 391.8+ KB


# Tratamento dos dados e exportação para o bucket

In [7]:
# fazendo um merge entre os dataframes de estados e municipios
df = df_municipios.merge(df_estados[['uf', 'codigo_uf']], on='codigo_uf')
df

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario,uf
0,5200050,Abadia de Goiás,-16.7573,-49.4412,0,52,1050,62,America/Sao_Paulo,GO
1,5200100,Abadiânia,-16.1970,-48.7057,0,52,9201,62,America/Sao_Paulo,GO
2,5200134,Acreúna,-17.3960,-50.3749,0,52,9645,64,America/Sao_Paulo,GO
3,5200159,Adelândia,-16.4127,-50.1657,0,52,9769,64,America/Sao_Paulo,GO
4,5200175,Água Fria de Goiás,-14.9778,-47.7823,0,52,9771,62,America/Sao_Paulo,GO
...,...,...,...,...,...,...,...,...,...,...
5565,3306107,Valença,-22.2445,-43.7129,0,33,5921,24,America/Sao_Paulo,RJ
5566,3306156,Varre-Sai,-20.9276,-41.8701,0,33,2917,22,America/Sao_Paulo,RJ
5567,3306206,Vassouras,-22.4059,-43.6686,0,33,5923,24,America/Sao_Paulo,RJ
5568,3306305,Volta Redonda,-22.5202,-44.0996,0,33,5925,24,America/Sao_Paulo,RJ


In [8]:
# iterar dataframe por uf
# para cada linha filtrada do dataframe enviar para para a respectiva pasta com o nome da uf no bucket

for uf in df['uf'].unique():
    df_tmp = df.loc[ df['uf'] == uf ]
    
    #armazenando a variavel df_temp em buffer
    csv_buffer = StringIO()
    df_tmp.to_csv(csv_buffer)
    
    client.put_object(Body=csv_buffer.getvalue(), Bucket='municipios-estados', Key=f'{uf}/cidades.csv')